In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, avg,udf, month,max, datediff, lit, to_date,count, rank
from pyspark.sql.window import Window
from pyspark.sql.types import StringType


In [2]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

23/09/06 10:22:34 WARN Utils: Your hostname, ubuntu-Lenovo-Legion-5-15ARH05 resolves to a loopback address: 127.0.1.1; using 172.16.5.112 instead (on interface wlp4s0)
23/09/06 10:22:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 10:22:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Reading csv**


In [3]:
#reading csv file
listing_df = spark.read.csv('cleaned_data/clean_listing_csv/', header=True, inferSchema=True)
review_df = spark.read.csv('cleaned_data/clean_reviews_csv/', header=True, inferSchema=True)
calender_df = spark.read.csv('cleaned_data/clean0_calendar_csv/', header=True, inferSchema=True)


Question 1: Listings with the Highest Average Price:
Using groupBy and orderBy, find the top 10 listings in Boston with the highest average price per night. Rename the columns to "Listing ID" and "Average Price."


In [21]:

# Group by listing ID and calculate the average price per night
average_price_df = listing_df.groupBy('city').agg(avg('price').alias('Average Price'))

# Order by average price in descending order and select the top 10
top_10_listings = average_price_df.orderBy(col('Average Price').desc()).limit(10)

# Rename the columns
top_10_listings = top_10_listings.withColumnRenamed('id', 'Listing_ID')

# Show the result
top_10_listings.show()

+--------------------+------------------+
|                city|     Average Price|
+--------------------+------------------+
|    South End Boston|           45000.0|
|             ALLSTON|           29500.0|
|         Charlestown|27164.285714285714|
|           Cambridge|           21200.0|
|   Roslindale Boston|           20000.0|
|       Jamaica Plain|18441.176470588234|
|              Boston|18248.658843252306|
|         East Boston|           16950.0|
|Jamaica Plain (Bo...|           15000.0|
|           Brookline|14784.615384615385|
+--------------------+------------------+



Question 2:  UDF for Room Type Analysis: Create a UDF to categorize listings based on their room type and whether they are hosted by superhosts. Apply this UDF to listings_df to analyze room type preferences among superhosts.

In [30]:
# Define the UDF to categorize listings
def categorize_room_type(is_superhost, room_type):
    if is_superhost:
        return f'Superhost - {room_type}'
    else:
        return f'Non-Superhost - {room_type}'

# Register the UDF
categorize_room_udf = udf(categorize_room_type, StringType())

# Apply the UDF to listings_df
listing_df_2 = listing_df.withColumn(
    'room_type_category',
    categorize_room_udf(col('host_is_superhost'), col('room_type'))
)

# Analyze room type preferences among superhosts
room_type_preferences = listing_df_2.groupBy('room_type_category').count()

# Show the result
room_type_preferences.show()

+--------------------+-----+
|  room_type_category|count|
+--------------------+-----+
|Superhost - Entir...|  190|
|Non-Superhost - P...|  740|
|Superhost - Priva...|  149|
|Non-Superhost - E...| 1397|
|Superhost - Share...|    8|
|Non-Superhost - S...|   44|
+--------------------+-----+



Question 3: To create a User-Defined Function (UDF) that converts extracted months from integers to their corresponding month names

In [7]:
# Define a UDF to convert month integers to month names
def int_to_month_name(month_int):
    month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    return month_names[month_int - 1] if 1 <= month_int <= 12 else None

# Register the UDF with Spark
month_name_udf = udf(int_to_month_name, StringType())

# Apply the UDF to the DataFrame to create a new column
df_with_month_name = calender_df.withColumn("month_name", month_name_udf(month(col('date'))))

# Show the result
df_with_month_name.show()


+----------+----------+---------+-----+----------+
|listing_id|      date|available|price|month_name|
+----------+----------+---------+-----+----------+
|  12147973|2017-09-05|    false| null|       Sep|
|  12147973|2017-09-04|    false| null|       Sep|
|  12147973|2017-09-03|    false| null|       Sep|
|  12147973|2017-09-02|    false| null|       Sep|
|  12147973|2017-09-01|    false| null|       Sep|
|  12147973|2017-08-31|    false| null|       Aug|
|  12147973|2017-08-30|    false| null|       Aug|
|  12147973|2017-08-29|    false| null|       Aug|
|  12147973|2017-08-28|    false| null|       Aug|
|  12147973|2017-08-27|    false| null|       Aug|
|  12147973|2017-08-26|    false| null|       Aug|
|  12147973|2017-08-25|    false| null|       Aug|
|  12147973|2017-08-24|    false| null|       Aug|
|  12147973|2017-08-23|    false| null|       Aug|
|  12147973|2017-08-22|    false| null|       Aug|
|  12147973|2017-08-21|    false| null|       Aug|
|  12147973|2017-08-20|    fals

Question 3 - Busiest Booking Month:
Using the date column in the calendar_df, determine which month in Boston had the highest number of bookings. We can use aggregation functions like groupBy and count.


In [9]:
# Filter for Boston listings (you can adjust the city name as needed)
boston_calendar = calender_df.join(listing_df, calender_df.listing_id == listing_df.id, "inner")

# Extract the month from the 'date' column
boston_calendar = boston_calendar.withColumn('booking_month', month_name_udf(month(col('date'))))
# Filter the DataFrame to include only rows where "available" is False
booked_calendar = boston_calendar.filter(col('available') == 'False')

# Group by the booking month and count the number of bookings
booking_counts = booked_calendar.groupBy('booking_month').count()

# Find the month with the highest number of bookings
busiest_month = booking_counts.orderBy(col('count').desc()).first()

# Show the result
print(f"The busiest booking month in Boston is month {busiest_month['booking_month']} with {busiest_month['count']} bookings.")

The busiest booking month in Boston is month Sep with 50042 bookings.


Question 4: Calculate the number of days since the last booking for each listing in Boston. Use a Window Function to find listings that have been vacant for the longest period.


In [18]:
# Filter calendar_df to include only Boston listings
boston_calendar = calender_df.join(listing_df.select('id'), calender_df.listing_id == listing_df.id, "inner")
booked_calendar = boston_calendar.filter(col('available') == 'False')

# # Calculate the maximum date for each listing
window_spec = Window.partitionBy('listing_id').orderBy(col('date').desc())
max_date_df = booked_calendar.select('listing_id', 'date').withColumn('max_date', max('date').over(window_spec))

# min_date_df = max_date_df.select('listing_id', 'date','max_date').withColumn('min_date', min('date').over(window_spec))


# # # Get the maximum date as a separate variable
# max_date = max_date_df.select(max_date_df['max_date']).first()[0]

# # Calculate the number of days since the last booking
vacant_listings = max_date_df.withColumn('days_since_last_booking', datediff(col('max_date'), col('date')))

# # Show the result
vacant_listings.show()

+----------+----------+----------+-----------------------+
|listing_id|      date|  max_date|days_since_last_booking|
+----------+----------+----------+-----------------------+
|      3353|2016-12-30|2016-12-30|                      0|
|      3353|2016-12-29|2016-12-30|                      1|
|      3353|2016-12-28|2016-12-30|                      2|
|      3353|2016-12-27|2016-12-30|                      3|
|      3353|2016-12-26|2016-12-30|                      4|
|      3353|2016-12-25|2016-12-30|                      5|
|      3353|2016-12-24|2016-12-30|                      6|
|      3353|2016-12-23|2016-12-30|                      7|
|      3353|2016-12-22|2016-12-30|                      8|
|      3353|2016-12-21|2016-12-30|                      9|
|      3353|2016-12-20|2016-12-30|                     10|
|      3353|2016-12-19|2016-12-30|                     11|
|      3353|2016-12-18|2016-12-30|                     12|
|      3353|2016-12-17|2016-12-30|                     1

Question 5 - find the top 5 hosts in Boston with the highest average nightly price for their listings, along with the number of reviews they've received and the average number of bathrooms for their listings,

In [80]:
# Filter listings for Boston
boston_listings = listing_df.filter(col('city') == 'Boston')

# Calculate the average nightly price for each listing and the total number of reviews for each host
listing_avg_price_reviews = boston_listings.groupBy('host_name').agg(
    avg(col('price').cast("float")).alias('avg_nightly_price'),
    count('number_of_reviews').alias('total_reviews'),
    avg(col('bathrooms')).alias('avg_bathrooms')  # Calculate the average number of bathrooms
)

# Use window functions to rank hosts by average nightly price
window_spec = Window.orderBy(col('avg_nightly_price').desc())
ranked_hosts = listing_avg_price_reviews.withColumn('rank', rank().over(window_spec))

# Filter and select the top 5 hosts with the highest average nightly price
top_5_hosts = ranked_hosts.filter(col('rank') <= 5).select('host_name', 'avg_nightly_price', 'total_reviews', 'avg_bathrooms')

# Show the result
top_5_hosts.show()

+---------+-----------------+-------------+-------------+
|host_name|avg_nightly_price|total_reviews|avg_bathrooms|
+---------+-----------------+-------------+-------------+
| Vladimir|         400000.0|            1|          1.0|
|   Nastia|         125000.0|            1|          1.0|
|    Stacy|          80000.0|            1|          2.5|
|   Hylton|          75000.0|            1|          3.5|
|   Lively|          75000.0|            1|          1.0|
+---------+-----------------+-------------+-------------+



23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 17:10:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/05 1


Question 6 - Superhosts by Property Type:
Pivot the data to create a table that shows the count of superhosts and non-superhosts for each property type in Boston. Rows represent property types, and columns represent counts of superhosts and non-superhosts.

In [86]:
# Filter listings for Boston
boston_listings = listing_df.filter(col('city') == 'Boston')

# Pivot the data to count superhosts and non-superhosts for each property type
pivot_table = boston_listings.groupBy('property_type') \
    .pivot('host_is_superhost', ['True', 'False']) \
    .count() \
    .fillna(0)

# Rename the columns for clarity
pivot_table = pivot_table.withColumnRenamed('True', 'Superhost Count') \
    .withColumnRenamed('False', 'Non-Superhost Count')

# Show the result
pivot_table.show()


+---------------+---------------+-------------------+
|  property_type|Superhost Count|Non-Superhost Count|
+---------------+---------------+-------------------+
|      Apartment|            205|               1529|
|      Townhouse|             11|                 24|
|Bed & Breakfast|              8|                 23|
|           null|              0|                  2|
|           Loft|              5|                 21|
|     Guesthouse|              0|                  1|
|          Villa|              1|                  2|
|   Entire Floor|              0|                  2|
|          Other|              2|                 10|
|           Dorm|              0|                  1|
|    Condominium|             21|                137|
|          House|             65|                304|
|           Boat|              1|                 11|
+---------------+---------------+-------------------+

